In [ ]:
! pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install -q openai

In [ ]:
import os
from dotenv import load_dotenv
from google.colab import userdata
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api')


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# **Query Classification**

In [ ]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Phân loại truy vấn của người dùng theo từng loại phù hợp"""

    datasource: Literal["chatting", "real-time", "academy"] = Field(
        ...,
        description="Với câu hỏi của người dùng, hãy phân loại lĩnh vực nào có liên quan nhất để trả lời câu hỏi của họ",
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """Bạn là chuyên gia trong việc phân loại câu hỏi của người dùng thuộc lĩnh vực nào.
Dựa vào chủ đề mà câu hỏi của người dùng đang đề cập đến, hãy phân loại chúng thuộc lĩnh vực nào

Ví dụ: Câu hỏi: Bây giờ là mấy giờ?
Câu trả lời: đây là câu hỏi thuộc dạng real-time"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router
router = prompt | structured_llm

In [ ]:
question = """Xin chào buổi sáng, bạn ổn chứ?"""

result = router.invoke({"question": question})
result

RouteQuery(datasource='chatting')

In [ ]:
question = """Nhiệt độ ngay bây giờ tại Việt Nam là bao nhiêu độ C?"""

result = router.invoke({"question": question})
result

RouteQuery(datasource='real-time')

In [ ]:
question = """Làm thế nào để một hàm trong lập trình python?"""

result = router.invoke({"question": question})
result

RouteQuery(datasource='academy')

# **Sematic Routing**

In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Two prompts
physics_template = """Bạn là một giáo sư vật lý rất thông minh. \
Bạn rất giỏi trả lời các câu hỏi về vật lý một cách ngắn gọn và dễ hiểu. \
Khi bạn không biết câu trả lời cho một câu hỏi, bạn thừa nhận rằng bạn không biết.

Đây là một câu hỏi:
{query}"""

math_template = """Bạn là một nhà toán học rất giỏi. Bạn rất giỏi trong việc trả lời các câu hỏi toán học. \
Bạn rất giỏi vì bạn có thể chia nhỏ các bài toán khó thành các thành phần, \
trả lời các thành phần, rồi ghép chúng lại với nhau để trả lời câu hỏi rộng hơn.

Đây là một câu hỏi:
{query}"""

# Embed prompts
embeddings = HuggingFaceEmbeddings(model_name= "hiieu/halong_embedding")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt
    print("Lĩnh Vực Toán" if most_similar == math_template else "Lĩnh Vực Vật Lý")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)



In [ ]:
print(chain.invoke("Thuyết tương đối là gì"))

Lĩnh Vực Vật Lý
Thuyết tương đối là một lý thuyết vật lý do Albert Einstein phát triển, bao gồm hai phần chính: thuyết tương đối hẹp và thuyết tương đối rộng.

1. **Thuyết tương đối hẹp** (1905) tập trung vào các hệ quy chiếu chuyển động với vận tốc không đổi. Nó khẳng định rằng các định luật vật lý là giống nhau trong mọi hệ quy chiếu quán tính và rằng tốc độ ánh sáng trong chân không là hằng số, không phụ thuộc vào chuyển động của nguồn sáng hay người quan sát. Một trong những hệ quả nổi bật của thuyết này là sự tương đương giữa khối lượng và năng lượng, được thể hiện qua công thức nổi tiếng \(E=mc^2\).

2. **Thuyết tương đối rộng** (1915) mở rộng khái niệm này để bao gồm cả các hệ quy chiếu có gia tốc và mô tả trọng lực như là sự cong của không-thời gian do sự hiện diện của khối lượng. Nó giải thích các hiện tượng như sự lệch hướng của ánh sáng khi đi qua trường hấp dẫn và sự chuyển động của các hành tinh.

Tóm lại, thuyết tương đối đã cách mạng hóa cách chúng ta hiểu về không gian,

In [ ]:
print(chain.invoke("Cách tính diện tích của hình hộp chữ nhật trong hình học không gian"))

Lĩnh Vực Toán
Để tính diện tích của hình hộp chữ nhật (hay còn gọi là hình chữ nhật 3 chiều), chúng ta cần hiểu rõ các thành phần của nó. Hình hộp chữ nhật có 6 mặt, mỗi mặt là một hình chữ nhật. Diện tích bề mặt của hình hộp chữ nhật được tính bằng cách cộng diện tích của tất cả các mặt.

### Bước 1: Xác định kích thước của hình hộp chữ nhật
Giả sử hình hộp chữ nhật có chiều dài \( l \), chiều rộng \( w \), và chiều cao \( h \).

### Bước 2: Tính diện tích của từng mặt
Hình hộp chữ nhật có 6 mặt, và diện tích của mỗi mặt được tính như sau:
- 2 mặt có diện tích \( l \times w \) (mặt trên và mặt dưới)
- 2 mặt có diện tích \( l \times h \) (mặt trước và mặt sau)
- 2 mặt có diện tích \( w \times h \) (mặt bên trái và mặt bên phải)

### Bước 3: Tính tổng diện tích bề mặt
Diện tích bề mặt tổng cộng \( A \) của hình hộp chữ nhật được tính bằng công thức:
\[
A = 2(l \times w + l \times h + w \times h)
\]

### Ví dụ
Giả sử một hình hộp chữ nhật có chiều dài \( l = 4 \) cm, chiều rộng \( w = 3 